### Черненко Сергей Евгеньевич (184-321)

# Нейронная сеть и PyTorch
Порядок выполнения:
1. Используя фреймворк PyTorch создать нейронную сеть;
2. Загрузить базу данных MNIST;
3. Разделить данные на выборки: обучающую, проверочную, тестовую;
4. Провести предварительную обработку данных;
5. Обучить нейронную сеть;
6. Подобрать гиперпараметры;
7. Оценить результаты на тестовой выборке.


In [170]:
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets
import torchvision.transforms as transforms

import time

## Загрузка базы MNIST

In [171]:
# количество образцов для загрузки 
batch_size = 20

transform = transforms.ToTensor()
X_train = datasets.MNIST(root='data', train=True,
                                   download=True, transform=transform)
Y_train = datasets.MNIST(root='data', train=False,
                                  download=True, transform=transform)

# Делим на образцы
X_train = torch.utils.data.DataLoader(X_train, batch_size=batch_size)
Y_train = torch.utils.data.DataLoader(Y_train, batch_size=batch_size)



## Объявляем архитектуру сети

In [172]:
class ClassPT(nn.Module):
    def __init__(self):
        super(ClassPT, self).__init__()
        
        #первый входной слов
        self.fc1 = nn.Linear(28*28, 50)
        
        #скрытый слой
        self.fc2 = nn.Linear(50, 10)
                                      
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        
        x = x.view(-1, 28 * 28)
        
        x = F.sigmoid(self.fc1(x))
        x = F.sigmoid(self.fc2(x))
        
        
        return x

model = ClassPT()

## Тестирования сети на тестовой выборке

In [173]:
def test_database():
    ind = 0
    class_correct = list(0. for i in range(10))
    class_total = list(0. for i in range(10))
    model.eval() 
    for data, target in Y_train:
        ind += 1
        if ind < 300:
            # вычисление прогнозируемых результатов
            output = model(data)
            loss = criterion(output, target)
            _, pred = torch.max(output, 1)
            # сравниваем прогнозы с желаемыми значениями
            correct = np.squeeze(pred.eq(target.data.view_as(pred)))
            for i in range(batch_size):
                label = target.data[i]
                class_correct[label] += correct[i].item()
                class_total[label] += 1

    print('Точность на тестовой выборке - ',np.sum(class_correct)/np.sum(class_total))

## Обучение сети

In [174]:
# функция потерь (softmax)
criterion = nn.CrossEntropyLoss() 

# функция оптимизации
optimizer = torch.optim.SGD(model.parameters(), lr=0.05)

#  колличество эпох
n_epochs = 20

model.train()
for epoch in range(n_epochs):
    start_time = time.clock()
    train_loss = 0.0
    for data, target in X_train:
        optimizer.zero_grad()
        # вычисление прогнозируемых результатов
        output = model(data)
        # вычисление ошибки
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()*data.size(0)
    train_loss = train_loss/len(X_train.dataset)
    print('Эпоха: {0}, Training Loss: {1}, Время на одну эпоху -  {2}'.format(epoch+1, train_loss, round((time.clock() - start_time),2)))
    test_database()


Эпоха: 1, Training Loss: 2.0622503256003064, Время на одну эпоху -  13.42
Точность на тестовой выборке -  0.76438127090301
Эпоха: 2, Training Loss: 1.7692794627348583, Время на одну эпоху -  17.28
Точность на тестовой выборке -  0.8103678929765886
Эпоха: 3, Training Loss: 1.6905324353377025, Время на одну эпоху -  14.69
Точность на тестовой выборке -  0.8389632107023411
Эпоха: 4, Training Loss: 1.653626064459483, Время на одну эпоху -  15.38
Точность на тестовой выборке -  0.8583612040133779
Эпоха: 5, Training Loss: 1.6319425425926843, Время на одну эпоху -  12.87
Точность на тестовой выборке -  0.8690635451505017
Эпоха: 6, Training Loss: 1.6165193337996802, Время на одну эпоху -  14.53
Точность на тестовой выборке -  0.8714046822742475
Эпоха: 7, Training Loss: 1.604848760565122, Время на одну эпоху -  14.58
Точность на тестовой выборке -  0.8739130434782608
Эпоха: 8, Training Loss: 1.5955770066579182, Время на одну эпоху -  14.64
Точность на тестовой выборке -  0.8822742474916387
Эпох

## Тестирование сети на проверочной выборке

In [167]:
ind = 0
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
model.eval() 
for data, target in Y_train:
    ind += 1
    if ind > 300:
        # вычисление прогнозируемых результатов
        output = model(data)
        loss = criterion(output, target)
        _, pred = torch.max(output, 1)
        # сравниваем прогнозы с желаемыми значениями
        correct = np.squeeze(pred.eq(target.data.view_as(pred)))
        for i in range(batch_size):
            label = target.data[i]
            class_correct[label] += correct[i].item()
            class_total[label] += 1

print('Точность на проверочной выборке - ',np.sum(class_correct)/np.sum(class_total))

Точность на проверочной выборке -  0.93875
